In [17]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import selenium.webdriver
from datetime import date
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from dateutil.relativedelta import relativedelta
import csv

In [19]:
def scrap(origin,destin,trDate):
    global none
    baseDataUrl = "https://www.makemytrip.com/flight/search?itinerary="+ origin +"-"+ destin +"-"+ trDate +"&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E"

    try:
        driver = webdriver.Chrome(r"C:\Users\WIN-10\Downloads\chromedriver.exe") 
        print ("Requesting URL: " + baseDataUrl)

        driver.get(baseDataUrl)  
        print ("Webpage found ...")

        element_xpath = '//*[@id="left-side--wrapper"]/div[2]'

        
        element = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.XPATH, element_xpath)))

        print ("Scrolling document upto bottom ...")
        for j in range(1, 100):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        
        body = driver.find_element_by_tag_name("body").get_attribute("innerHTML")

        print("Closing Chrome ...")
        driver.quit()

        print("Getting data from DOM ...")
        soupBody = BeautifulSoup(body)
        
        spanFlightName = soupBody.find_all("span", 'boldFont blackText airlineName')
        print("spanFlightName  = " + str(len(spanFlightName)))

        Time = soupBody.find_all("p", {"class":"blackText fontSize18 blackFont appendBottom2 makeFlex hrtlCenter"})			# Tags with Departure Time
        print("Time Count in Scraping  = " + str(len(Time)))

        City = soupBody.findAll("p", {"class": "darkText"})
        print("City Count in Scraping  = " + str(len(City)))

        spanFlightCost = soupBody.findAll("p", {"class": "blackText fontSize18 blackFont white-space-no-wrap"})			# Tags with Flight Cost
        print("spanFlightCost Count in Scraping  = " + str(len(spanFlightCost)))

        pFlightStops = soupBody.findAll("p", {"class": "flightsLayoverInfo"})
        print("pFlightStops Count in Scraping  = " + str(len(pFlightStops)))
        
        transform(spanFlightName,trDate,City,Time,spanFlightCost,pFlightStops)



   
    except :
        PrintException()
        print(trDate," scraping complete")
        
        none =True 

In [20]:
dates=(pd.date_range(start='18/02/2022', end='24/02/2022'))
New_date=[]
for i,date in enumerate(dates):
    day=str(date.day).zfill(2)   
    month=str(date.month).zfill(2)
    year=str(date.year)
    date=day+'/'+month+'/'+year  
    New_date.append(date)
print(New_date)

['18/02/2022', '19/02/2022', '20/02/2022', '21/02/2022', '22/02/2022', '23/02/2022', '24/02/2022']


In [21]:
#Lists to store scraped data
Airline=[]
Journey_date=[]
From=[]
To=[]
Price=[]
Dtime=[]
Atime=[]
Stops=[]

In [22]:
flight_data = pd.DataFrame(np.nan, index=[0], columns=['Airline', 'Journey_date','From','To','Dtime','Atime','Stops','Price'])
flight_data.to_csv('Flight.csv')
 
def transform(spanFlightName,trDate,City,Time,spanFlightCost,pFlightStops):
    
    Airline.clear()
    Journey_date.clear()
    Price.clear()
    Stops.clear()
    From.clear()
    Dtime.clear()
    To.clear()
    Atime.clear()
    
    global flight_data
    print("Transaform Function Running.... ")
    for i in range(0,len(spanFlightName)):
        
        Airline.append(spanFlightName[i].string)
        Journey_date.append(trDate)
        Price.append(spanFlightCost[i].string)
        Stops.append(pFlightStops[i].string)
        
      
    for j in range(0,len(Time)):
        
        if(j%2==0):
            
            From.append(City[j].string)
            Dtime.append(Time[j].string)
        else:
            
            To.append(City[j].string)
            Atime.append(Time[j].string)
    temp = {'Airline':Airline,
            'Journey_date':Journey_date,
            'From':From,
            'To':To,
            'Dtime':Dtime,
            'Atime':Atime,
            'Stops':Stops,
            'Price':Price}
    df=pd.DataFrame(temp)
    df.to_csv('Flight.csv', mode='a', header=False)
    flight_data = pd.concat([flight_data,df])
    flight_data=flight_data.reset_index(drop=True)
    display(flight_data)
    print('Lenght : ',len(flight_data))

In [23]:
list_of_dest=['GOI','AMD','TIR','BLR','BOM','DEL']
for i in range(0,len(list_of_dest)):
    orgin=list_of_dest[i]
    for_to=list_of_dest
    print("--------------")
    for j in range(0,len(for_to)):
        destin=for_to[j]
        none = False
        print("origin = "+orgin)
        print("destin = "+destin+'\n')
        if orgin!=destin:
            for i,date in enumerate(New_date) :
                if none !=True:
                    scrap(orgin,destin,date)
                    print(" ")
                else:
                    break
            else: 
                print( "-----Skiped-----")
                print(" ")


--------------
origin = GOI
destin = GOI

origin = GOI
destin = AMD



C:\Users\WIN-10\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


Requesting URL: https://www.makemytrip.com/flight/search?itinerary=GOI-AMD-18/02/2022&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
Webpage found ...
Scrolling document upto bottom ...


C:\Users\WIN-10\anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 21
Time Count in Scraping  = 42
City Count in Scraping  = 42
spanFlightCost Count in Scraping  = 21
pFlightStops Count in Scraping  = 21
Transaform Function Running.... 


...

Requesting URL: https://www.makemytrip.com/flight/search?itinerary=DEL-BOM-24/02/2022&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
Webpage found ...
Scrolling document upto bottom ...


C:\Users\WIN-10\anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 86
Time Count in Scraping  = 172
City Count in Scraping  = 173
spanFlightCost Count in Scraping  = 86
pFlightStops Count in Scraping  = 86
Transaform Function Running.... 


,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SpiceJet,18/02/2022,Goa,Ahmedabad,06:20,08:20,Non stop,"₹ 5,315"
2,Go First,18/02/2022,Goa,Ahmedabad,20:40,22:15,Non stop,"₹ 5,315"
3,Go First,18/02/2022,Goa,Ahmedabad,14:00,20:55,1 stop via Bengaluru,"₹ 5,315"
4,IndiGo,18/02/2022,Goa,Ahmedabad,23:25,01:00,Non stop,"₹ 5,316"
...,...,...,...,...,...,...,...,...
6353,SpiceJet,24/02/2022,New Delhi,Mumbai,07:45,12:00,1 stop via Jabalpur,"₹ 8,264"
6354,Go First,24/02/2022,New Delhi,Mumbai,16:40,21:25,1 stop via Ahmedabad,"₹ 9,448"
6355,Air India,24/02/2022,New Delhi,Mumbai,13:15,17:50,1 stop via Udaipur,"₹ 9,735"
6356,IndiGo,24/02/2022,New Delhi,Mumbai,05:15,10:05,1 stop via Ahmedabad,"₹ 9,861"


Lenght :  6358
 
-----Skiped-----
 
origin = DEL
destin = DEL



In [28]:
flight_data.head(5)

,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SpiceJet,18/02/2022,Goa,Ahmedabad,06:20,08:20,Non stop,"₹ 5,315"
2,Go First,18/02/2022,Goa,Ahmedabad,20:40,22:15,Non stop,"₹ 5,315"
3,Go First,18/02/2022,Goa,Ahmedabad,14:00,20:55,1 stop via Bengaluru,"₹ 5,315"
4,IndiGo,18/02/2022,Goa,Ahmedabad,23:25,01:00,Non stop,"₹ 5,316"


In [30]:
##saving data frame as excelfile
flight_data.to_excel('E:\\flight_data.xlsx')